In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import numpy as np
import pandas as pd
import copy
from operator import itemgetter
import time

In [0]:
#Local
#file_path = "Term 6/01.112 - ML/Project/EN/" 


#Hosted
file_path = "/content/drive/My Drive/Machine Learning project/FR/"

train_file_name = "train"
test_file_in = "dev.in"
p2_test_file_out = "dev.p2.out"
p3_test_file_out = "dev.p3.out"
p4_test_file_out = "dev.p4.out"
p5_test_file_out = "dev.p5.out"
blind_test_file_in = "test.in"  #CHANGE TO test2.in
blind_test_file_out = "test.p5.out"  #CHANGE TO test2.p5.out

# Part 2

Read and parse train file

In [0]:
file = open(file_path + train_file_name,"r", encoding='utf8')  #Open training file
l = [line.rstrip('\n').split(" ") for line in file]  #Remove newline character and split words and tags

#Adds Stop and Start tags for each sentence
m = [['__Start__',"Start"]]  #First Start tag
 
for pair in l:
  if len(pair) == 1:
    m.append(['__Stop__',"Stop"])
    m.append(['__Start__',"Start"])
  else:
    m.append(pair)

m =  m[:-1]  #Drops last extra Start tag

df = pd.DataFrame(m)  #convert Array to pandas DataFrame

In [0]:
tags = df[1].unique()  #Obtain all unique tags in train
words = df[0].unique()  #Obtain all unique words in train

tagc_dict = df[1].value_counts().to_dict()  #Dictionary of tag counts

#Sorting tags in descending order
tags = np.sort(tags)  
tags = tags[::-1]

Count words

In [0]:
#Creating a dict with count of words dataframe
#Dict -> key = tags, values = count of words df for tag
#Df -> column name = tag, index = word

wordc_dictdf = {}  #Dictionary with word count dataframe
for tag in tags:
    temp_df = df.loc[df[1] == tag][0].value_counts().to_frame()
    temp_df.columns = [tag]
    wordc_dictdf[tag] = temp_df

Calculate emission parameter

In [0]:
#Keeps count of each word and creates new dict for emission parameter calculation
emm_dictdf = copy.deepcopy(wordc_dictdf)  

k=1
#Calculating emission parameter for each word
for tag in tags:
  emm_dictdf[tag] /= (tagc_dict[tag]+k) # Count (u -> v)/ Count (u)

Functions to obtain tag from word

In [0]:
#Smoothing function if test word not in train
def smoothing(tag,k=1):  # k=1 - Hyperparameter for smoothing
  return k/(tagc_dict[tag]+k)

#Returns emission parameter value of a word given a tag
def emission(word,tag):
  if word in words:
    return emm_dictdf[tag][tag].get(word,0)
  else:
    return smoothing(tag)

#Find tag with maximum emission parameter for a word
def argmax_em(word):
  tmpemm_dict={}
  for tag in tags:
    tmpemm_dict.update({tag:emission(word,tag)})
  return max(tmpemm_dict, key = lambda x: tmpemm_dict.get(x))  

read dev.out and write dev.p2.out

In [0]:
#Reading test file and predicting tag given word
test_file = open(file_path+test_file_in,"r", encoding='utf8')  #Open training file
l = [line.rstrip('\n').split(" ") for line in test_file]  #Remove newline character and split words and tags

out = []
for word in l:
  if word[0] == '':
    out.append('\n')
  else:
    out.append(word[0] + " " + argmax_em(word[0]) + '\n')
    
    
with open(file_path + p2_test_file_out, 'w', encoding='utf8') as f:
  for item in out:
    f.write('%s' %(item))

# Part 3

Transition parameter calculation

In [0]:
#Creating empty dataframe to hold all transition parameter values
#u is an array to store possible tags except stop
u = ["Start"]
u += [tag for tag in tags if (tag != 'Stop' and tag != 'Start')]

v = u[1:] + ['Stop']
v = np.asarray(v)

u= np.asarray(u)

tran_df = pd.DataFrame(0,index=v, columns=u)

#Calculating count from u to v
for num in range(len(df[1])):
  if (df[1][num] != 'Stop'):
    tran_df[df[1][num]][df[1][num+1]] +=1
    
#Calculating transition parameter    
for tag in u:
  tran_df[tag] /= tagc_dict[tag]

read and parse dev.in for Part 3

In [0]:
#Preparing input file for Viterbi Calculation

test_file = open(file_path+test_file_in,"r", encoding='utf8')  #Open training file
l = [line.rstrip('\n').split(" ") for line in test_file]  #Remove newline character and split words and tags

m_test = ['__Start__']  #First Start tag

#Put Start and Stop tag placeholders and track index of each placeholder
indexing = [0]
for i in range(len(l)):
  if l[i][0] == '':
    m_test = m_test + ['__Stop__']
    indexing += [len(m_test)-1]
    m_test = m_test + ['__Start__']
    indexing += [len(m_test)-1]
  else:
    m_test = m_test + l[i]
    
#Clean up extra placeholders
m_test =  m_test[:-1]  #Drops last extra Start tag
indexing = indexing[:-1]  #Drops last extra Start tag

#Creating easy sequencing index to obtain each word sequence
index = [indexing[0::2],indexing[1::2]]

#Store each sequence of words for Viterbi Algorithm
sequence = []
for i in range(len(index[0])):
  sequence += [m_test[index[0][i]:index[1][i]:]]
  
# sequence is a colelction of the tweets to be passed to viterbi

Viterbi Algorithm Implementation

And 

write dev.p3.out

In [0]:
output = []

for s in range(len(sequence)):
  n = len(sequence[s])
  
  #Empty Dataframe
  val_pi = pd.DataFrame(index=range(n),columns=u)
  tag_pi = [None] * (n+1)
  
  #Base Case
  for v_tag in tags[tags != 'Stop']:
    if v_tag == 'Start':
      val_pi[v_tag][0] = 1
    else:
      val_pi[v_tag][0] = 0
  
  #Recursive Case
  for k in range(1,n):  
    for v_tag in v[v != 'Stop']:
      emm_v = emission(sequence[s][k],v_tag)
      if emm_v != 0:
        temp = []  
        for u_tag in u:
          temp.append(val_pi[u_tag][k-1] * tran_df[u_tag][v_tag] * emm_v)
        max_val = max(filter(lambda x: not np.isnan(x), temp))
        val_pi[v_tag][k] = max_val
    if val_pi.sum(axis=1)[k] != 0:        
      val_pi.loc[k,:] = val_pi.loc[k,:].div(val_pi.sum(axis=1)[k], axis=0)    
  
  #Finding Tags
  tag_pi[n] = 'Stop'
  for i in range(n-1,-1,-1):
    temp=[]
    for v_tag in tags[tags != 'Stop']:
      temp.append([v_tag,val_pi[v_tag][i] * tran_df[v_tag][tag_pi[i+1]]])
    tag_pi[i] = max(filter(lambda x: not np.isnan(x[1]), temp), key=itemgetter(0))[0]
  
  #Create output List
  for i in range(1,len(sequence[s])):
    output.append(sequence[s][i] + " " +tag_pi[i] + '\n')
  output.append('\n')
  

#Write to File  
with open(file_path + p3_test_file_out, 'w', encoding='utf8') as f:
  for item in output:
    f.write('%s' %(item))  

# Part 4 

Second order transition parameter calculation

In [0]:
#Second order assumption transition parameter calculation

#DataFrame to represent tags in y(i-2) and y(i-1) as index
tran_df2 = pd.DataFrame(index=u, columns=u)

#DataFrame to represent tags in y(i) as index
initial = pd.DataFrame(columns = v)
initial.loc[0] = 0.0

#Deep copy of y(i) DataFrame
for tag1 in u:
  for tag2 in u:
    tran_df2[tag1][tag2] = copy.deepcopy(initial)
    
#Count( (y(i-2), y(i-1)); y(i) )    
for i in range(len(df[1])-1):
  j = i if df[1][i] == 'Start' else i-1
  if df[1][j] != 'Stop' and df[1][i] != 'Stop': 
    tran_df2[df[1][j]][df[1][i]][df[1][i+1]] += 1.0

#Divide by Count( (y(i-2), y(i-2)) )
for tag1 in u:
  for tag2 in u:
    sum_row = tran_df2[tag1][tag2].sum(axis=1)[0]
    if sum_row != 0:
      tran_df2[tag1][tag2] /= sum_row    

#a(t, u, v) => a(y(i-2), y(i-1), y(i)) => tran_df2[y(i-2)][y(i-1)][y(i)][0]
#tran_df2[u][u][v][0]      

Second order Viterbi Algorithm Implementation

And

write dev.p4.out 

In [0]:
output = []
for s in range(len(sequence)):

  n = len(sequence[s]) # n=16 | range(n) = 0,1,2,...,15
    
  #Empty Dataframe
  val_pi2 = pd.DataFrame(index=u,columns=range(n+1))

  initial = pd.DataFrame(columns = tags)
  initial.loc[0] = None

  for k in range(n+1):
    for t_tag in u:
      val_pi2[k][t_tag] = copy.deepcopy(initial)
      
  #pi(k, (u,v)) => val_pi2[k][u][v]
  #val_pi2[range(n+1)][u][tags][0]


  #Base Case
  for k in range(2):
    for u_tag in u:
      for v_tag in tags:
        if k == 0:
          if u_tag == 'Start' and v_tag == 'Start':
            val_pi2[k][u_tag][v_tag][0] = 1.0
          else:
            val_pi2[k][u_tag][v_tag][0] = 0.0
        else:
          if u_tag == 'Start' and v_tag != 'Start':
            val_pi2[k][u_tag][v_tag][0] = tran_df2['Start']['Start'][v_tag][0] * emission(sequence[s][k], v_tag)
          else:
            val_pi2[k][u_tag][v_tag][0] = 0.0       
            
  #val_pi2 [range(n+1)][u][tags][0]
  #tran_df2[    u     ][u][ v  ][0]


  #Recursive Case
  for k in range(2,n):  #2,3,...,15
    for v_tag in v[v != 'Stop']:
      emm_v = emission(sequence[s][k],v_tag)
      if emm_v != 0:
        
        for u_tag in u[u != 'Start']:      
          temp = []  
          for t_tag in u:          
            temp.append(val_pi2[k-1][t_tag][u_tag][0] * tran_df2[t_tag][u_tag][v_tag][0] * emm_v)

          filtered = list(filter(lambda x: not np.isnan(x),temp))
          max_val = float('NaN') if len(filtered) == 0 else max(filtered)
          val_pi2[k][u_tag][v_tag][0] = max_val
    
    for u_tag in u:
      val_sum = val_pi2[k][u_tag].sum(axis=1)[0]
      if val_sum != 0:  
        val_pi2[k][u_tag].loc[:] = val_pi2[k][u_tag].loc[:].div(val_sum)                    
        

  #Final Transition Case      
  for u_tag in u[u != 'Start']:
    temp = []
    for t_tag in u[u != 'Start']:
      temp.append(val_pi2[n-1][t_tag][u_tag][0] * tran_df2[t_tag][u_tag]['Stop'][0])
    filtered = list(filter(lambda x: not np.isnan(x), temp))
    max_val = float('NaN') if len(filtered) == 0 else max(filtered)
    val_pi2[n][u_tag]['Stop'] = max_val           
    

  #Finding Tags  
  tag_pi2 = [None] * (n+1)  

  tag_pi2[n] = 'Stop'
  tag_pi2[0] = 'Start'

  temp=[]
  for u_tag in u[u != 'Start']:
    temp.append([u_tag, val_pi2[n][u_tag]['Stop'][0]])
  tag_pi2[n-1] = max(filter(lambda x: not np.isnan(x[1]), temp), key=itemgetter(0))[0]

  for k in range(n-2,0,-1):
    temp=[]
    for t_tag in u[u!='Start']:
      temp.append([ t_tag, val_pi2[k+1][t_tag][tag_pi2[k+1]][0] * tran_df2[t_tag][tag_pi2[k+1]][tag_pi2[k+2]][0] ])
    tag_pi2[k] = max(filter(lambda x: not np.isnan(x[1]), temp), key=itemgetter(0))[0]    
    
    
  #Create output List
  for i in range(1,len(sequence[s])):
    output.append(sequence[s][i] + " " + tag_pi2[i] + '\n')
  output.append('\n')  
  

#Write to File  
with open(file_path + p4_test_file_out, 'w') as f:
  for item in output:
    f.write('%s' %(item))  

# Part 5

Improvement

In [0]:
my_out = open(file_path + p3_test_file_out, 'r', encoding='utf8')
actual_out = open(file_path + 'dev.out', 'r', encoding='utf8')

pred_out = [line.rstrip('\n').split(" ") for line in my_out]
compare_out = [line.rstrip('\n').split(" ") for line in actual_out]

In [0]:
for i in range(len(pred_out)):  
  if len(pred_out[i]) != 1: #Ignore all the blank lines
    if pred_out[i][1] != compare_out[i][1]: #If predicted not same as actual
      
      #Update emission parameters 
      #Word in train list of words, increase emission probability of the correct tag
      if pred_out[i][0] in words: 
        if pred_out[i][0] in emm_dictdf[compare_out[i][1]].index:
          emm_dictdf[compare_out[i][1]][compare_out[i][1]][pred_out[i][0]] += 2
        else:
          emm_dictdf[compare_out[i][1]].loc[pred_out[i][0]] = 2
      #Word not in train list of words, add to list of words and set emission probability for correct tag
      else: 
        emm_dictdf[compare_out[i][1]][compare_out[i][1]][pred_out[i][0]] = 10
        np.append(words,pred_out[i][0])
        #tagc_dict[compare_out[i][1]] -= 1
        
      #Update Part 3 transition parameters
      if i == 0 or len(pred_out[i-1]) == 1:
        tran_df['Start'][compare_out[i][1]] = tran_df['Start'][compare_out[i][1]] + 2
      else:
        tran_df[compare_out[i-1][1]][compare_out[i][1]] = tran_df[compare_out[i-1][1]][compare_out[i][1]] + 2
               

HMM Calculation

And

write dev.p5.out

In [0]:
output = []

for s in range(len(sequence)):
  n = len(sequence[s])
  
  #Empty Dataframe
  val_pi = pd.DataFrame(index=range(n),columns=u)
  tag_pi = [None] * (n+1)
  
  #Base Case
  for v_tag in tags[tags != 'Stop']:
    if v_tag == 'Start':
      val_pi[v_tag][0] = 1
    else:
      val_pi[v_tag][0] = 0
  
  #Recursive Case
  for k in range(1,n):  
    for v_tag in v[v != 'Stop']:
      emm_v = emission(sequence[s][k],v_tag)
      if emm_v != 0:
        temp = []  
        for u_tag in u:
          temp.append(val_pi[u_tag][k-1] * tran_df[u_tag][v_tag] * emm_v)
        max_val = max(filter(lambda x: not np.isnan(x), temp))
        val_pi[v_tag][k] = max_val
    if val_pi.sum(axis=1)[k] != 0:        
      val_pi.loc[k,:] = val_pi.loc[k,:].div(val_pi.sum(axis=1)[k], axis=0)    
  
  #Finding Tags
  tag_pi[n] = 'Stop'
  for i in range(n-1,-1,-1):
    temp=[]
    for v_tag in tags[tags != 'Stop']:
      temp.append([v_tag,val_pi[v_tag][i] * tran_df[v_tag][tag_pi[i+1]]])
    tag_pi[i] = max(filter(lambda x: not np.isnan(x[1]), temp), key=itemgetter(0))[0]
  
  #Create output List
  for i in range(1,len(sequence[s])):
    output.append(sequence[s][i] + " " +tag_pi[i] + '\n')
  output.append('\n')
  

#Write to File  
with open(file_path + p5_test_file_out, 'w', encoding='utf8') as f:
  for item in output:
    f.write('%s' %(item))  

test.in/test2.in parsing

In [0]:
#Preparing input file for Viterbi Calculation

test_file = open(file_path+blind_test_file_in,"r", encoding='utf8')  #Open training file
l = [line.rstrip('\n').split(" ") for line in test_file]  #Remove newline character and split words and tags

m_test = ['__Start__']  #First Start tag

#Put Start and Stop tag placeholders and track index of each placeholder
indexing = [0]
for i in range(len(l)):
  if l[i][0] == '':
    m_test = m_test + ['__Stop__']
    indexing += [len(m_test)-1]
    m_test = m_test + ['__Start__']
    indexing += [len(m_test)-1]
  else:
    m_test = m_test + l[i]
    
#Clean up extra placeholders
m_test =  m_test[:-1]  #Drops last extra Start tag
indexing = indexing[:-1]  #Drops last extra Start tag

#Creating easy sequencing index to obtain each word sequence
index = [indexing[0::2],indexing[1::2]]

#Store each sequence of words for Viterbi Algorithm
sequence = []
for i in range(len(index[0])):
  sequence += [m_test[index[0][i]:index[1][i]:]]
  
# sequence is a collection of the tweets to be passed to viterbi

HMM calculation

And

write test.p5.out/test2.p5.out

In [0]:
output = []

for s in range(len(sequence)):
  n = len(sequence[s])
  
  #Empty Dataframe
  val_pi = pd.DataFrame(index=range(n),columns=u)
  tag_pi = [None] * (n+1)
  
  #Base Case
  for v_tag in tags[tags != 'Stop']:
    if v_tag == 'Start':
      val_pi[v_tag][0] = 1
    else:
      val_pi[v_tag][0] = 0
  
  #Recursive Case
  for k in range(1,n):  
    for v_tag in v[v != 'Stop']:
      emm_v = emission(sequence[s][k],v_tag)
      if emm_v != 0:
        temp = []  
        for u_tag in u:
          temp.append(val_pi[u_tag][k-1] * tran_df[u_tag][v_tag] * emm_v)
        max_val = max(filter(lambda x: not np.isnan(x), temp))
        val_pi[v_tag][k] = max_val
    if val_pi.sum(axis=1)[k] != 0:        
      val_pi.loc[k,:] = val_pi.loc[k,:].div(val_pi.sum(axis=1)[k], axis=0)    
  
  #Finding Tags
  tag_pi[n] = 'Stop'
  for i in range(n-1,-1,-1):
    temp=[]
    for v_tag in tags[tags != 'Stop']:
      temp.append([v_tag,val_pi[v_tag][i] * tran_df[v_tag][tag_pi[i+1]]])
    tag_pi[i] = max(filter(lambda x: not np.isnan(x[1]), temp), key=itemgetter(0))[0]
  
  #Create output List
  for i in range(1,len(sequence[s])):
    output.append(sequence[s][i] + " " +tag_pi[i] + '\n')
  output.append('\n')
  

#Write to File  
with open(file_path + blind_test_file_out, 'w', encoding='utf8') as f:
  for item in output:
    f.write('%s' %(item))  